In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')


In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [4]:
df.shape

(103, 3)

# PART 2

In [5]:
!wget -q -O "loc_data.csv" https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv
    
loc_data = pd.read_csv("loc_data.csv")
loc_data=loc_data.rename(columns={"Postal Code": "PostalCode"})
loc_data.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
df=pd.merge(df,loc_data, on = "PostalCode" )
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# Part 3

In [7]:
df[["Borough","Neighborhood"]].groupby("Borough").count()


,Neighborhood
Borough,
Central Toronto,9
Downtown Toronto,17
Downtown Toronto Stn A,1
East Toronto,4
East Toronto Business,1
East York,4
East York/East Toronto,1
Etobicoke,11
Etobicoke Northwest,1


### Let's analysis North York borough

In [8]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

In [9]:
NY_data = df[df['Borough'] == 'North York'].reset_index(drop=True)
NY_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [10]:
address = 'North York, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [11]:
# create map of North York using latitude and longitude values
map_NY = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(NY_data['Latitude'], NY_data['Longitude'], NY_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NY)  
    
map_NY

In [35]:
# The code was removed by Watson Studio for sharing.

In [13]:
#searching for top 200 venues in each neighborhoods

def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

NY_venues=getNearbyVenues(NY_data.Neighborhood,NY_data.Latitude, NY_data.Longitude, radius=1000)

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills North
Glencairn
Don Mills South
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview East
York Mills, Silver Hills
Downsview West
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview Central
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [14]:
NY_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
3,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store
4,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant


In [15]:
# create map of North York venues
map_NY_venues = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(NY_venues['Venue Latitude'], NY_venues['Venue Longitude'], NY_venues['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NY_venues)  
    
map_NY_venues

In [16]:
NY_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",32,32,32,32,32,32
Bayview Village,17,17,17,17,17,17
"Bedford Park, Lawrence Manor East",39,39,39,39,39,39
Don Mills North,30,30,30,30,30,30
Don Mills South,41,41,41,41,41,41
Downsview Central,4,4,4,4,4,4
Downsview East,22,22,22,22,22,22
Downsview Northwest,33,33,33,33,33,33
Downsview West,9,9,9,9,9,9


In [17]:
print('There are {} uniques categories.'.format(len(NY_venues['Venue Category'].unique())))

There are 143 uniques categories.


In [18]:
# one hot encoding
NY_onehot = pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NY_onehot['Neighborhood'] = NY_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [NY_onehot.columns[-1]] + list(NY_onehot.columns[:-1])
NY_onehot = NY_onehot[fixed_columns]
# frequency of occurrence of each category
NY_grouped = NY_onehot.groupby('Neighborhood').mean().reset_index()
NY_grouped

,Neighborhood,ATM,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,...,Theater,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00
1,Bayview Village,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00
2,"Bedford Park, Lawrence Manor East",0.000000,0.00000,0.000000,0.025641,0.00000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.00000,0.00
3,Don Mills North,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.033333,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00
4,Don Mills South,0.000000,0.00000,0.000000,0.024390,0.02439,0.000000,0.024390,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02439,0.00
5,Downsview Central,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.00000,0.00
6,Downsview East,0.000000,0.00000,0.045455,0.000000,0.00000,0.000000,0.000000,0.045455,0.00,...,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.045455,0.000000,0.00000,0.00
7,Downsview Northwest,0.000000,0.00000,0.000000,0.030303,0.00000,0.000000,0.000000,0.030303,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.00000,0.00
8,Downsview West,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.00000,0.00
9,"Fairview, Henry Farm, Oriole",0.000000,0.00000,0.000000,0.022727,0.00000,0.000000,0.022727,0.000000,0.00,...,0.022727,0.022727,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.00000,0.00


In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

#  new dataframe
NY_venues_sorted = pd.DataFrame(columns=columns)
NY_venues_sorted['Neighborhood'] = NY_grouped['Neighborhood']

for ind in np.arange(NY_grouped.shape[0]):
    NY_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)

NY_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Pizza Place,Park,Coffee Shop,Ski Chalet,Gas Station,Middle Eastern Restaurant,Shopping Mall,Frozen Yogurt Shop,Fried Chicken Joint
1,Bayview Village,Intersection,Japanese Restaurant,Gas Station,Bank,Park,Restaurant,Café,Chinese Restaurant,Skating Rink,Shopping Mall
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sushi Restaurant,Bank,Fast Food Restaurant,Sandwich Place,Café,Greek Restaurant,Butcher,Comfort Food Restaurant
3,Don Mills North,Japanese Restaurant,Coffee Shop,Burger Joint,Pizza Place,Restaurant,Cafeteria,Café,Caribbean Restaurant,Sandwich Place,Dessert Shop
4,Don Mills South,Restaurant,Gym,Coffee Shop,Supermarket,Sporting Goods Shop,Hockey Arena,Burger Joint,History Museum,Dim Sum Restaurant,Grocery Store
5,Downsview Central,Vietnamese Restaurant,Restaurant,Baseball Field,ATM,New American Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Moving Target
6,Downsview East,Coffee Shop,Turkish Restaurant,Italian Restaurant,Latin American Restaurant,Middle Eastern Restaurant,Soccer Field,Chinese Restaurant,Sandwich Place,Gas Station,Park
7,Downsview Northwest,Hotel,Grocery Store,Discount Store,Fast Food Restaurant,Gas Station,Coffee Shop,Pizza Place,Pharmacy,Sandwich Place,Fried Chicken Joint
8,Downsview West,Park,Bank,Shopping Mall,Vietnamese Restaurant,Coffee Shop,Pizza Place,Grocery Store,Moving Target,Miscellaneous Shop,Mobile Phone Shop
9,"Fairview, Henry Farm, Oriole",Coffee Shop,Clothing Store,Fast Food Restaurant,Bank,Bakery,Sandwich Place,Japanese Restaurant,Juice Bar,Restaurant,Shopping Mall


In [20]:
#Clustering

#!pip install scikit-learn

In [21]:
from sklearn.cluster import KMeans

NY_grouped_clustering = NY_grouped.drop('Neighborhood', axis=1)

# k-means clustering
kmeans = KMeans(n_clusters=5, random_state=0).fit(NY_grouped_clustering)


#kmeans.labels_[0:10] 

In [22]:
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 3, 2, 2, 0, 2], dtype=int32)

In [23]:
NY_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

NY_merged = NY_data

NY_merged = NY_merged.join(NY_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

NY_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Park,Shopping Mall,Pharmacy,Bus Stop,Skating Rink,Pizza Place,Fast Food Restaurant,Cosmetics Shop,Convenience Store,Fish & Chips Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Coffee Shop,Sporting Goods Shop,Pizza Place,Portuguese Restaurant,Lounge,Park,Golf Course,Men's Store,Playground,Gym / Fitness Center
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2,Furniture / Home Store,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Sushi Restaurant,Fried Chicken Joint,Vietnamese Restaurant,Dessert Shop,Boutique
3,M3B,North York,Don Mills North,43.745906,-79.352188,2,Japanese Restaurant,Coffee Shop,Burger Joint,Pizza Place,Restaurant,Cafeteria,Café,Caribbean Restaurant,Sandwich Place,Dessert Shop
4,M6B,North York,Glencairn,43.709577,-79.445073,2,Grocery Store,Fast Food Restaurant,Coffee Shop,Furniture / Home Store,Gas Station,Pharmacy,Pet Store,Restaurant,Mediterranean Restaurant,Electronics Store


In [28]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NY_merged['Latitude'], NY_merged['Longitude'], NY_merged['Neighborhood'], NY_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters